In [176]:
import pandas as pd

In [177]:
df = pd.read_csv("All orders - short custom description.csv")
df = df[(df['date'] >= '2021-01-01') & (df['date'] <= '2023-12-31')]
df = df[(df['order_state'] == 'Order Dispatched') | (df['order_state'] == 'Order Refunded')]
df = df[df['attribute_summary'] != 'Size: ladies']
df = df[df['attribute_summary'] != 'Size: One Size']
df = df[df['Channel'] == 'Leather Company']

df.rename(columns={'attribute_summary': 'Size'}, inplace=True)
temp_df = df['Size'].str.split(': ', expand=True)
temp_df.columns = ['F_Size', 'Size']
df['Size'] = temp_df['Size']

df = df.sort_values(by='date').reset_index(drop=True)
df['colour'] = df['colour'].str.capitalize()
df.tail()

C:\Users\retai\AppData\Local\Temp\ipykernel_10740\1653511506.py:1: DtypeWarning: Columns (1,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("All orders - short custom description.csv")


,order_date,order_id,quantity,order_state,grand_total_inc,reference,price_inc,date,Channel,title,...,manufacturer_name,Size,customs_description,colour,F_Cat,S_Cat,T_Cat,Category,Order fees total amount,time
21037,2023-12-31 10:09:05,75289,1,Order Dispatched,151.583,3777902,149.00,2023-12-31,Leather Company,Twin Zip Fastening Leather Jacket Cognac : Ciara,...,ASHWOOD,10,Sale,Cognac,Sale,NaN,NaN,NaN,NaN,NaN
21038,2023-12-31 01:38:47,75283,1,Order Dispatched,189.000,2518707,189.00,2023-12-31,Leather Company,Ladies Leather Biker Jacket Black : Serefina,...,ASHWOOD,20,Ladies > Leather Jackets,Black,Ladies > Leather Jackets,NaN,NaN,Ladies > Ladies Jackets & Coats,NaN,NaN
21039,2023-12-31 22:54:41,75328,1,Order Dispatched,143.100,1365005,159.00,2023-12-31,Leather Company,Leather Biker Jacket Timber : Bronx,...,ASHWOOD,L,Mens > Leather Jackets,Timber,Mens > Leather Jackets,NaN,NaN,Mens > Leather Jackets,NaN,NaN
21040,2023-12-31 13:12:14,75301,1,Order Dispatched,134.100,1367807,149.00,2023-12-31,Leather Company,Leather Jacket Tan : Edinburgh,...,ASHWOOD,2XL,Mens > Leather Jackets,Tan,Mens > Leather Jackets,NaN,NaN,Mens > Leather Jackets,NaN,NaN
21041,2023-12-31 18:45:29,75321,1,Order Dispatched,31.981,2908406,29.99,2023-12-31,Leather Company,00029084,...,ASHWOOD,XL,Mens > Accessories,NaN,Mens > Accessories,NaN,NaN,Mens > Accessories,NaN,NaN


In [178]:
# df[df['customs_description'] == 'Mens > Leather Jackets']['attribute_summary'].value_counts()
temp_men = df[df['customs_description'] == 'Mens > Leather Jackets']['Size'].value_counts().to_frame('count').join(df[df['customs_description'] == 'Mens > Leather Jackets']['Size'].value_counts(normalize=True).to_frame('%'))
temp_men['%'] = 100 * temp_men['%']
temp_men['%'] = temp_men['%'] / 2
temp_men['%'] = temp_men['%'].round(0)
temp_men['%'] = temp_men['%'].astype(int)
temp_men['%']

Size
L      10
M       9
XL      8
2XL     6
S       5
3XL     4
4XL     3
5XL     2
XS      1
6XL     1
7XL     0
8XL     0
XXS     0
9XL     0
Name: %, dtype: int32

In [179]:
# df[df['customs_description'] == 'Ladies > Leather Jackets']['attribute_summary'].value_counts()
temp_ladies = df[df['customs_description'] == 'Ladies > Leather Jackets']['Size'].value_counts().to_frame('count').join(df[df['customs_description'] == 'Ladies > Leather Jackets']['Size'].value_counts(normalize=True).to_frame('%'))
temp_ladies['%'] = 100 * temp_ladies['%']
temp_ladies['%'] = temp_ladies['%'] / 2
temp_ladies['%'] = temp_ladies['%'].round(0)
temp_ladies['%'] = temp_ladies['%'].astype(int)
temp_ladies['%']

Size
14    10
12    10
16     8
10     7
18     6
20     4
8      3
22     1
24     1
26     0
6      0
Name: %, dtype: int32

In [180]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date').reset_index(drop=True)
df['Month'] = df['date'].dt.month_name()
temp = df[df['customs_description'] == 'Mens > Leather Jackets'].groupby(['Month', 'colour'])['quantity'].sum().reset_index()
temp.sort_values(by='quantity', ascending=False)

,Month,colour,quantity
44,December,Black,306
194,November,Black,264
63,December,Tan,190
213,November,Tan,172
221,October,Black,164
...,...,...,...
138,June,Cream,1
54,December,Gray,1
139,June,Ginger,1
121,July,Ginger,1


In [181]:
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
temp = df[df['customs_description'] == 'Mens > Leather Jackets']
temp = temp.groupby(['colour', 'Month'])['quantity'].size().groupby(
    level=1).nlargest(5).reset_index(-1, drop=True).reindex(new_order, level=0)
# temp = temp.to_frame()
temp.to_excel('Mens Leather Jackets.xlsx')

In [182]:
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
temp = df[df['customs_description'] == 'Ladies > Leather Jackets']
temp = temp.groupby(['colour', 'Month'])['quantity'].size().groupby(
    level=1).nlargest(5).reset_index(-1, drop=True).reindex(new_order, level=0)
# temp = temp.to_frame()
temp.to_excel('Ladies Leather Jackets.xlsx')

In [183]:
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
temp = df[df['customs_description'] == 'Mens > Leather Jackets']
temp = temp.groupby(['Size', 'Month'])['quantity'].size().groupby(
    level=1).nlargest(5).reset_index(-1, drop=True).reindex(new_order, level=0)
temp.to_excel('Size Mens Leather Jackets.xlsx')

In [184]:
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
temp = df[df['customs_description'] == 'Ladies > Leather Jackets']
temp = temp.groupby(['Size', 'Month'])['quantity'].size().groupby(
    level=1).nlargest(5).reset_index(-1, drop=True).reindex(new_order, level=0)
temp.to_excel('Size Ladies Leather Jackets.xlsx')